# STUMP Tiles Algorithm for Cache Optimization

This tutorial summarizes the modified STUMP algorithm for optimizing cache misses, implemented in the `stumpy.stump_tiles` module in the STUMPY library. This algorithm was also heavily influenced by the approach outlined in Section 4.2.2 (Algorithm 2) of [Time Series Analysis with Matrix Profile on HPC Systems](https://mediatum.ub.tum.de/download/1471292/1471292.pdf) and the tiling scheme outlined in Section 3.2 of [Scaling Time Series Motif Discovery with GPUs](https://www.cs.ucr.edu/~eamonn/public/GPU_Matrix_profile_VLDB_30DraftOnly.pdf).

## Getting Started

Let's import a few packages we'll need to analyze and visualize our data and our algorithm.

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from stumpy import stump#, stump_tiles

## Introduction

The STUMP algorithm is already well-optimized to compute the Matrix Profile extremely fast. The methodologies outlined in the [Matrix Profile I](https://www.cs.ucr.edu/~eamonn/PID4481997_extend_Matrix%20Profile_I.pdf) and [Matrix Profile II](https://www.cs.ucr.edu/~eamonn/STOMP_GPU_final_submission_camera_ready.pdf) papers are designed to theoretically as perform as efficiently as possible.

So why exactly do we have to work on improving the already-optimized STUMP algorithm? Where exactly is our room for improvement? This is a question we will answer in this document, and we'll return to it once we have explored a practical phenomenon of computer systems.

### Effect of Cache on Performance

Let's consider an array with, say, 10,000,000,000 elements (i.e. 10 billion). Suppose now that we want to divide the array into 100,000 equal groups, each of which will contain 100,000 elements. And finally suppose we want to iterate over the first group, i.e. the first 100,000 elements, and fetch those values.

In [2]:
# number of elements
n = 10000000000
# number of groups
n_groups = 100000
# array of n elements
arr = np.zeros(n, dtype=np.int64)

We want to iterate over the first 100,000 elements sequentially, so we generate the indices of iteration.

In [3]:
# generate sequential indices
iter_range = list(range(n // n_groups))
# print number of indices
print(len(iter_range))
# print first 10 indices
print(iter_range[:10])

100000
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


There are 100,000 indices and they are sequential. Looks good! Let's use our indices to iterate over our array.

In [4]:
%%timeit

# iterate over indices & access array values
for i in iter_range:
    arr[i]

7.61 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Alright great! Let's make a note of the time this process took.

Now we would like to change up our iteration a bit. Suppose now we want to iterate over the first element of every group of 1000 elements and double those values. So our indices will be 0, 100000, 200000 ... and so on. Let's accordingly generate our indices of iteration again.

In [5]:
# generate indices that are equal distance apart
iter_range = list(range(0, n, n_groups))
# print number of indices
print(len(iter_range))
# print first 10 indices
print(iter_range[:10])

100000
[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000]


Looks good! Once again we have 100,000 indices of iteration. Let's use our indices to iterate over our array again.

In [6]:
%%timeit

# iterate over indices & access array values
for i in iter_range:
    arr[i]

30.1 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Notice how this iteration took several milliseconds more to perform the same task the exact same number of times. The only difference were the indices of iteration. Why is there such a significant difference between the performance of the two loops? The answer has to do with **cache memory.**

### What is Cache?

Have you ever had to fix something in your house with a hammer, to which end you ran down to your garage, grabbed the entire toolbox that contains the hammer and ran back up to your room? You were nearly certain that all you needed was a hammer, but you brought the whole toolbox full of screw drivers and wrenches anyway, just in case you need them? If a scenario like this sounds familiar, then you already understand how cache memory works.

Cache is a very fast, compact memory storage where the computer temporarily stores data that is frequently fetched or is likely to be fetched in the near future. Just as you may fetch your hammer and similar tools from your garage and keep them with you until your job is done, the computer will fetch data from its main memory and store it in the cache while it is being read frequently. You can learn more about different levels of cache and how they work in this [article](https://searchstorage.techtarget.com/definition/cache).

Going back to our exploration above, we can explain the difference in iteration performance using the cache. When we access our array by index using `arr[i]`, the CPU doesn't just fetch the value at the $i^{th}$ index. Instead, it actually fetches a whole chunk of memory, typically of 64 bytes, that includes the value at the $i^{th}$ index, as well as a whole set of values that sit around the $i^{th}$ memory block. These chunks of memory are called **cache lines.** Fetching a cache line takes relatively more time, but once a cache line is captured on the cache, it takes exponentially less time for the device to fetch data from within the cache line.

In our first example, the indices were sequential. This means that for the first element, the CPU fetches the first cache line. After that, since the indices are sequential, the next few indices are already pre-fetched in our cache line. In our second example, the indices were evenly-spaced out. This means that for every iteration, the CPU had to fetch a whole new cache line (provided a single cache line stores less memory than the amount of memory used to store each group of array values). This is why our second example took more time to complete than our first.

<img alt="Cache Lines Visualization" src="images/cache_lines.gif" width="900" />

### What's in it for STUMPY?

Now that we understand how better caching and utilization of cache lines can improve performance, we can bring STUMPY into the picture. Because STUMPY's functions are computation-heavy and generally involve transformations of large scale time series data, there is a significant cost of constantly indexing and reading the values in that data. We will explore how the STUMP algorithm works and how we can improve it to leverage cache lines in the next section.

## How does STUMP work?

The STUMP algorithm takes in a time series and a window size `m` as parameters, and computes the **Matrix Profile**. In order to do this, it computes the **Pairwise Euclidean Distance Matrix** for all sliding window pairs. To learn more about the Matrix Profile, see [The Matrix Profile Tutorial](https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html).

Each element in the computed distance matrix represents each different possible sliding window pair. This means the matrix will have dimensions `(n_A - m + 1) x (n_B - m + 1)`, where `n_A` and `n_B` are the lengths of our time series' and `m` is the window size.

For example, if we perform a self-join with a time series that has 20 elements and our window size is 5, we get a `16 x 16` distance matrix.

<img alt="Distance Matrix" src="images/distance_matrix_example.png" width="560" />

The exclusion zone for the distance matrix should be at `±2` (`np.ceil(5 / 4)`).

<img alt="Distance Matrix with Exclusion Zone" src="images/distance_matrix_example_exclusion.png" width="600" />

In order to optimize the computation, the STUMP algorithm computes the distances in a diagonal manner. This way, distance values can be propagated along diagonals, which can improve the efficiency of computation.

<img alt="Distance Matrix Traversing" src="images/distance_matrix_example_traverse.gif" width="600" />

To learn more about why this is efficient, see the [Matrix Profile I](https://www.cs.ucr.edu/~eamonn/PID4481997_extend_Matrix%20Profile_I.pdf) paper.

Now let's apply what we learnt about cache lines. It should be obvious that the traversal of the matrix in diagonals is not the most cache-friendly of methods. 2-dimensional arrays are usually stored in the computer's memory one row after another, so in every step of the diagonal we are jumping forward by `n + 1` memory blocks. Even if our cache system is smart enough to fetch memory blocks that are below our selected block of memory in our matrix, we are still getting a lot of cache misses as we move down the diagonal.

<img alt="Distance Matrix Traversing Cache Misses" src="images/distance_matrix_example_cache_misses.gif" width="600" />

As demonstrated above, we have to fetch a new cache line several times while traversing a diagonal. The cache line is represented by the orange rectangle, and every time it repositions, we are experiencing a cache miss.

How can we improve this by leveraging the cache memory, while continuing to use the diagonal traversal method (as opposed to brute force calculation)? That's where STUMP tiles comes in!

## How does STUMP tiles work?

The STUMP Tiles algorithm is an adaptation of the STUMP algorithm to leverage cache lines when computing the distance matrix, and it's based on two principles:

- Dividing the matrix into tiles
- Traversing multiple diagonals simultaneously

### Division of the Distance Matrix into Tiles

In order to prevent cache misses, we divide our matrix into tiles of a given pre-defined length.

<img alt="Distance Matrix Tiles" src="images/distance_matrix_example_tiles.png" width="600" />

In the example above, we divided our matrix into tiles of length 5 each. Note that the tiles near the edges do not have the same dimensions as the other tiles since they're near the edges. It's also noteworthy that, although we divided the entire matrix into tiles, not all tiles are to be used. Tiles that are entirely greyed out inside the exclusion zone have no computable distances inside them, and so we can ignore them.

Once we've outlined our tiles, the idea is to compute all distances within each tile before moving onto the next one.

<img alt="Distance Matrix Traversing Tiles" src="images/distance_matrix_example_tiles_traversal.gif" width="600" />

The philosopy behind doing so is that, values within a tile are more likely to be captured within the same cache line, and so it makes sense to compute them together. Note that we are still computing distances in diagonals within each tile.

Additionally, the handling of each tile does not need to be sequential within one thread. The tiles can be distributed into multiple threads, since each thread will have a cache memory of its own. It's important, however, to not split a single tile into two threads. Each thread must serve a whole number of tiles.

### Multiple Diagonal Traversal

Having divided our matrix into tiles, we would also like to traverse multiple diagonals at a time. Animated below is an example of how we would traverse 2 diagonals simultaneously within each tile.

<img alt="Distance Matrix Traversing Tiles with Multiple Diagonals" src="images/distance_matrix_example_multiple_diagonals.gif" width="600" />

Traversing multiple diagonals involves operating consecutively on horizontally aligned distances. It should be obvious that this also helps improve cache misses. The animation below demonstrates more closely the order of traversal within a single tile.

<img alt="=Multiple Diagonal Traversal in a Single Tile" src="images/multiple_diagonal_traversal.gif" width="200" />